<a href="https://colab.research.google.com/github/inrap8206/Dacon/blob/main/%5B%EB%8D%B0%EC%9D%B4%EC%BD%98%5D%EC%99%80%EC%9D%B8%ED%92%88%EC%A7%88%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [데이콘] 와인품질분류
- https://dacon.io/competitions/open/235610/overview/description

# Baseline

In [27]:
import pandas as pd
from google.colab import drive, files
drive.mount('/content/drive')
import warnings
warnings.filterwarnings(action='ignore')
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

train = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/train.csv')
test = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/test.csv')
sub = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/sample_submission.csv')


# 변주형 범수 처리
train['type'] = train['type'].map(lambda x : 1 if x == 'white' else 0)
test['type'] = test['type'].map(lambda x : 1 if x == 'white' else 0)

# 불필요 컬럼 삭제
label = train['quality']
train.drop(['index', 'quality'], axis=1, inplace=True)
test.drop('index', axis=1, inplace=True)


# # 피쳐 엔지니어링 (정제, 이상치처리, 파생변수 생성)
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()

# train = sc.fit_transform(train)
# test = sc.transform(test)


# 모델링
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

model = RandomForestClassifier(random_state=2022, n_estimators=200)

# 교차검증
score = cross_val_score(model, train, label, cv=5, scoring='accuracy')

print(score)
print(score.mean())


# 제출
model.fit(train, label)
preds = model.predict(test)
sub.iloc[:,1] = preds.round()
sub.to_csv('wine.csv', index=False)
files.download('wine.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[0.68545455 0.68909091 0.67060965 0.66515014 0.66060055]
0.6741811564231946


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Selection

## Manual

In [ ]:
!pip install catboost

In [3]:
from sklearn.linear_model import RidgeClassifier, LogisticRegression  
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

model_list = [RidgeClassifier(), LogisticRegression(), GaussianNB(), KNeighborsClassifier(n_neighbors=20), SVC(gamma = 0.01, C = 100, probability=True), 
              DecisionTreeClassifier(max_features=7 , max_depth=6,  min_samples_split=8), 
              RandomForestClassifier(random_state=2021), GradientBoostingClassifier(), AdaBoostClassifier(), XGBClassifier(), LGBMClassifier(), CatBoostClassifier(verbose=0)]
scores = []

for model in model_list:
  score = cross_val_score(model, train, label, cv=5, scoring='accuracy')
  scores.append(score.mean())


model_test = pd.DataFrame({'model':model_list, 'score':scores})
model_test.sort_values('score', ascending=False)

model     score
6   RandomForestClassifier(bootstrap=True, ccp_alp...  0.675455
11  <catboost.core.CatBoostClassifier object at 0x...  0.649624
10  LGBMClassifier(boosting_type='gbdt', class_wei...  0.633619
7   GradientBoostingClassifier(ccp_alpha=0.0, crit...  0.584135
9   XGBClassifier(base_score=0.5, booster='gbtree'...  0.578679
4   SVC(C=100, break_ties=False, cache_size=200, c...  0.563946
3   KNeighborsClassifier(algorithm='auto', leaf_si...  0.557393
1   LogisticRegression(C=1.0, class_weight=None, d...  0.545205
5   DecisionTreeClassifier(ccp_alpha=0.0, class_we...  0.533927
0   RidgeClassifier(alpha=1.0, class_weight=None, ...  0.531925
8   AdaBoostClassifier(algorithm='SAMME.R', base_e...  0.402040
2        GaussianNB(priors=None, var_smoothing=1e-09)  0.364752

## Pycatet

In [ ]:
!pip install pycaret

In [16]:
from pycaret.classification import *

train = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/train.csv')
setup_clf = setup(data=train, target='quality', session_id=2021, categorical_features=['type'], ignore_features=['index'])

Description                                     Value
0                               session_id                                      2021
1                                   Target                                   quality
2                              Target Type                                Multiclass
3                            Label Encoded  3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6
4                            Original Data                                (5497, 14)
5                           Missing Values                                     False
6                         Numeric Features                                        11
7                     Categorical Features                                         1
8                         Ordinal Features                                     False
9                High Cardinality Features                                     False
10                 High Cardinality Method                                      None
11                   Transformed Train Set                                (3847, 12)
12                    Transformed Test Set                                (1650, 12)
13                      Shuffle Train-Test                                      True
14                     Stratify Train-Test                                     False
15                          Fold Generator                           StratifiedKFold
16                             Fold Number                                        10
17                                CPU Jobs                                        -1
18                                 Use GPU                                     False
19                          Log Experiment                                     False
20                         Experiment Name                          clf-default-name
21                                     USI                                      dfb1
22                         Imputation Type                                    simple
23          Iterative Imputation Iteration                                      None
24                         Numeric Imputer                                      mean
25      Iterative Imputation Numeric Model                                      None
26                     Categorical Imputer                                  constant
27  Iterative Imputation Categorical Model                                      None
28           Unknown Categoricals Handling                            least_frequent
29                               Normalize                                     False
30                        Normalize Method                                      None
31                          Transformation                                     False
32                   Transformation Method                                      None
33                                     PCA                                     False
34                              PCA Method                                      None
35                          PCA Components                                      None
36                     Ignore Low Variance                                     False
37                     Combine Rare Levels                                     False
38                    Rare Level Threshold                                      None
39                         Numeric Binning                                     False
40                         Remove Outliers                                     False
41                      Outliers Threshold                                      None
42                Remove Multicollinearity                                     False
43             Multicollinearity Threshold                                      None
44             Remove Perfect Collinearity                                      True
45                              Clustering                                     False
46                    Clustering Ite

In [17]:
best_3 = compare_models(sort='Accuracy', n_select=3)

Model  Accuracy     AUC  Recall   Prec.  \
rf               Random Forest Classifier    0.6387  0.3263  0.3540  0.6396   
et                 Extra Trees Classifier    0.6356  0.3283  0.3624  0.6403   
lightgbm  Light Gradient Boosting Machine    0.6275  0.3128  0.3586  0.6232   
catboost              CatBoost Classifier    0.6215  0.3164  0.3586  0.6132   
gbc          Gradient Boosting Classifier    0.5737  0.3004  0.3072  0.5657   
dt               Decision Tree Classifier    0.5410  0.2591  0.3430  0.5442   
lda          Linear Discriminant Analysis    0.5378  0.2801  0.2571  0.5187   
ridge                    Ridge Classifier    0.5285  0.0000  0.2130  0.4513   
lr                    Logistic Regression    0.5256  0.2724  0.2187  0.4795   
qda       Quadratic Discriminant Analysis    0.4611  0.2719  0.3020  0.4869   
knn                K Neighbors Classifier    0.4580  0.2468  0.2383  0.4441   
ada                  Ada Boost Classifier    0.4133  0.2108  0.2149  0.3771   
nb                            Naive Bayes    0.3982  0.2524  0.2639  0.4368   
svm                   SVM - Linear Kernel    0.3775  0.0000  0.2006  0.3743   

              F1   Kappa     MCC  TT (Sec)  
rf        0.6221  0.4281  0.4346     0.921  
et        0.6213  0.4249  0.4306     0.718  
lightgbm  0.6144  0.4179  0.4214     0.759  
catboost  0.6094  0.4105  0.4133    15.498  
gbc       0.5555  0.3251  0.3314     4.265  
dt        0.5414  0.3176  0.3180     0.042  
lda       0.5122  0.2579  0.2659     0.024  
ridge     0.4596  0.2095  0.2288     0.017  
lr        0.4748  0.2143  0.2292     1.845  
qda       0.4598  0.2338  0.2403     0.021  
knn       0.4441  0.1543  0.1557     0.125  
ada       0.3861  0.1080  0.1142     0.242  
nb        0.4132  0.1464  0.1481     0.019  
svm       0.2953  0.1196  0.1637     0.130

In [18]:
tuned_best_3 = [tune_model(i) for i in best_3]
tuned_best_3

Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
0       0.6130  0.0000  0.3716  0.6082  0.5993  0.3915  0.3952
1       0.6026  0.0000  0.3433  0.6045  0.5879  0.3737  0.3785
2       0.6104  0.0000  0.3251  0.5989  0.5900  0.3859  0.3909
3       0.6052  0.7705  0.2747  0.5960  0.5843  0.3717  0.3783
4       0.5766  0.7690  0.2438  0.5367  0.5508  0.3262  0.3318
5       0.6130  0.7887  0.2729  0.5943  0.5896  0.3869  0.3918
6       0.6000  0.7697  0.2629  0.5672  0.5796  0.3706  0.3740
7       0.6198  0.0000  0.3362  0.6098  0.6004  0.4054  0.4087
8       0.6042  0.0000  0.3064  0.5821  0.5772  0.3654  0.3734
9       0.6562  0.0000  0.3253  0.6103  0.6279  0.4539  0.4603
Mean    0.6101  0.3098  0.3062  0.5908  0.5887  0.3831  0.3883
SD      0.0189  0.3795  0.0389  0.0221  0.0186  0.0309  0.0308

[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                        criterion='entropy', max_depth=10, max_features='log2',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0005, min_impurity_split=None,
                        min_samples_leaf=6, min_samples_split=5,
                        min_weight_fraction_leaf=0.0, n_estimators=250,
                        n_jobs=-1, oob_score=False, random_state=2021, verbose=0,
                        warm_start=False),
 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                      criterion='entropy', max_depth=10, max_features='log2',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0005, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
              

In [19]:
blender_best_3 = blend_models(estimator_list=tuned_best_3)

Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
0       0.5974  0.0000  0.3107  0.5941  0.5693  0.3480  0.3593
1       0.5896  0.0000  0.2808  0.5577  0.5569  0.3335  0.3469
2       0.6182  0.0000  0.3147  0.6237  0.5913  0.3800  0.3975
3       0.6182  0.7824  0.2663  0.6262  0.5883  0.3773  0.3950
4       0.5714  0.7732  0.2261  0.5384  0.5321  0.2981  0.3134
5       0.6208  0.7878  0.2687  0.6090  0.5921  0.3879  0.3996
6       0.5844  0.7661  0.2423  0.5590  0.5556  0.3272  0.3373
7       0.6172  0.0000  0.2920  0.5803  0.5818  0.3801  0.3909
8       0.6016  0.0000  0.2772  0.5626  0.5594  0.3485  0.3636
9       0.6562  0.0000  0.3166  0.6195  0.6232  0.4444  0.4575
Mean    0.6075  0.3109  0.2795  0.5871  0.5750  0.3625  0.3761
SD      0.0227  0.3809  0.0288  0.0302  0.0243  0.0385  0.0387

In [20]:
test = pd.read_csv('./drive/My Drive/Colab Notebooks/빅분기 실기/연습_데이콘_와인품질분류/test.csv')
test['type'] = test['type'].map(lambda x : 1 if x == 'white' else 0)
test.drop('index', axis=1, inplace=True)

final_model = finalize_model(blender_best_3)
prediction = predict_model(final_model, data=test)

In [24]:
sub['quality'] = prediction['Label']
sub.to_csv('wine_pycaret.csv', index=False)
files.download('wine_pycaret.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>